In [ ]:
!git clone -b anime https://github.com/karaage0703/tf-pose-estimation

Cloning into 'tf-pose-estimation'...
remote: Enumerating objects: 12, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 1496 (delta 4), reused 7 (delta 2), pack-reused 1484
Receiving objects: 100% (1496/1496), 98.35 MiB | 37.12 MiB/s, done.
Resolving deltas: 100% (812/812), done.


In [ ]:
!wget https://raw.githubusercontent.com/karaage0703/karaage_icon/master/karaage_icon.png

--2021-04-11 00:54:54--  https://raw.githubusercontent.com/karaage0703/karaage_icon/master/karaage_icon.png
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 78311 (76K) [image/png]
Saving to: ‘karaage_icon.png’

karaage_icon.png    100%[===================>]  76.48K  --.-KB/s    in 0.01s   

2021-04-11 00:54:54 (5.30 MB/s) - ‘karaage_icon.png’ saved [78311/78311]



In [ ]:
!wget https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml

--2021-04-11 00:56:33--  https://raw.githubusercontent.com/opencv/opencv/master/data/haarcascades/haarcascade_frontalface_alt.xml
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 676709 (661K) [text/plain]
Saving to: ‘haarcascade_frontalface_alt.xml’

haarcascade_frontal 100%[===================>] 660.85K  --.-KB/s    in 0.05s   

2021-04-11 00:56:33 (12.2 MB/s) - ‘haarcascade_frontalface_alt.xml’ saved [676709/676709]



In [ ]:
!sudo apt-get install -y swig
%cd /content/tf-pose-estimation/tf_pose/pafprocess
!swig -python -c++ pafprocess.i && python3 setup.py build_ext --inplace
!pip3 install slidingwindow
%cd /content/tf-pose-estimation/

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  swig3.0
Suggested packages:
  swig-doc swig-examples swig3.0-examples swig3.0-doc
The following NEW packages will be installed:
  swig swig3.0
0 upgraded, 2 newly installed, 0 to remove and 31 not upgraded.
Need to get 1,100 kB of archives.
After this operation, 5,822 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig3.0 amd64 3.0.12-1 [1,094 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 swig amd64 3.0.12-1 [6,460 B]
Fetched 1,100 kB in 1s (1,339 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 76, <> line 2.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: 

In [ ]:
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)

TensorFlow 1.x selected.
1.15.2


In [ ]:
import argparse
import logging
import sys
import time

from tf_pose import common
import cv2
import numpy as np
from tf_pose.estimator import TfPoseEstimator
from tf_pose.networks import get_graph_path, model_wh

import matplotlib.pyplot as plt

In [ ]:
import IPython
from google.colab import output

In [ ]:
cascade = cv2.CascadeClassifier('./haarcascade_frontalface_alt.xml')

In [ ]:
def run(img_str):
    # decode to image
    decimg = base64.b64decode(img_str.split(',')[1], validate=True)
    decimg = Image.open(BytesIO(decimg))
    decimg = np.array(decimg, dtype=np.uint8);
    decimg = cv2.cvtColor(decimg, cv2.COLOR_BGR2RGB)

    # pose estimation
    image_org = decimg.copy()
    humans = e.interence(decimg, resize_to_default=1, upsample_size=4)
    image_tmp = TfPoseEstimator.draw_humans(decimg, humans,
                                            imgcopy=False, mode='anime')
    out_img = face_overlay(image_org, image_tmp, cascade)

    # encode to string
    _, encimg = cv2.imencode(".jpg", out_img,
                             [int(cv2.IMWRITE_JPEG_QUALITY), 80])
    img_str = encimg.tostring()
    img_str = ("data:image/jpeg;base64," +
               base64.b64encode(img_str).decode('utf-8'))
    return IPython.display.JSON({'img_str': img_str})

output.register_callback('notebook.run', run)